In [1]:
import string
import types
import pandas as pd
import nltk
import numpy as np
from numpy.random import randn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.metrics import classification_report
import time
from sklearn import svm
import codecs, csv, sys
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB                                                                
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import accuracy_score
import itertools
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV

/Volumes/Barly/Anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# loading the csv containg embedding for test and train embedding data
df_train = pd.read_csv("/Volumes/Barly/NLP/Project/5k_testing/Type3/data/doc2Vec/train_doc2vec_pretrained.csv")
df_test = pd.read_csv("/Volumes/Barly/NLP/Project/5k_testing/Type3/data/doc2Vec/test_doc2vec_pretrained.csv")

In [3]:
# Concatenating the embeddings for question1 and question 2 and normalizing before passing it to classifiers
start_time = time.time()

train_embedding1 = map(np.fromstring, df_train['embedding_1'],\
                                      itertools.repeat(float, df_train.shape[0]),\
                                                       itertools.repeat(300, df_train.shape[0]))

train_embedding2 = map(np.fromstring, df_train['embedding_2'],\
                                      itertools.repeat(float, df_train.shape[0]),\
                                                       itertools.repeat(300, df_train.shape[0]))

X_label = np.array(df_train['is_duplicate'].tolist())
X_train = np.concatenate((np.array(train_embedding1), np.array(train_embedding2)), axis=1)
normalize = Normalizer().fit(X_train)
X_train = normalize.transform(X_train)

test_embedding1 = map(np.fromstring, df_test['embedding_1'],\
                                      itertools.repeat(float, df_test.shape[0]),\
                                                       itertools.repeat(300, df_test.shape[0]))

test_embedding2 = map(np.fromstring, df_test['embedding_2'],\
                                      itertools.repeat(float, df_test.shape[0]),\
                                                       itertools.repeat(300, df_test.shape[0]))

Y_label = np.array(df_test['is_duplicate'].tolist())
Y_train = np.concatenate((np.array(test_embedding1), np.array(test_embedding2)), axis=1) # concatenating the embedding for question 1 and question2
Y_train = normalize.transform(Y_train)

In [4]:
# Predicting is_duplicate value by passing embedding into shallow classifiers
def naive_bayes(x_train_matrix, x_test_matrix, y_train, y_test):    #NAIVE BAYERS MODEL                                                                
    nb = MultinomialNB(alpha=2.8)
    nb.fit(x_train_matrix, y_train)                                 #train the model
    y_pred = nb.predict(x_test_matrix)                              #make predictions for X_test
    print ("Naive Bayes Model score: " + str(accuracy_score(y_test, y_pred)))         
    print ("Naive Bayes Model confusion matrix:")
    print (confusion_matrix(y_test, y_pred))
    y_predicted_proba = nb.predict_proba(x_test_matrix)
    print log_loss(y_test, y_predicted_proba)
    return y_pred

def logistic_regression(x_train_matrix, x_test_matrix, y_train, y_test): 
    logReg = LogisticRegressionCV(Cs=[0.01, 0.1, 1, 10, 100], cv=5, solver='saga', n_jobs= -1 )
    logReg.fit(x_train_matrix, y_train)
    y_pred_log = logReg.predict(x_test_matrix)
    print ("Logistic Regression score: "+ str(accuracy_score(y_test, y_pred_log)))
    print ("Logistic Regression confusion matrix:")
    print (confusion_matrix(y_test, y_pred_log))
    y_predicted_proba = logReg.predict_proba(x_test_matrix)
    print log_loss(y_test, y_predicted_proba)
    return y_pred_log

def linear_svm(x_train_matrix, x_test_matrix, y_train, y_test):
    
    param_grid = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']}]
    svc = svm.SVC()
    svm1 = GridSearchCV(estimator=svc, param_grid=param_grid, n_jobs= -1, cv = 5) 
    svm1.fit(x_train_matrix, y_train)
    y_pred_svc = svm1.predict(x_test_matrix)
    print ("SVC score: " + str(accuracy_score(y_test, y_pred_svc)))
    print ("SVC confusionmatrix:")
    print (confusion_matrix(y_test, y_pred_svc))
    return y_pred_svc

In [5]:
preprocessing_time = time.time()
print("--- start -> preprocessing:%s seconds ---" % (preprocessing_time - start_time))


print ("\n\n Naive bayes result on doc2vec_embedding: ")
y_pred_nb = naive_bayes(X_train, Y_train, X_label, Y_label)
naive_bayes_time = time.time() 
print("--- preprocessing -> naive bayes :%s seconds ---" % (naive_bayes_time - preprocessing_time))


print ("-------------------------------------------")
print ("\n\n Logistic Regression result on doc2vec_embedding: ")
y_pred_lr = logistic_regression(X_train, Y_train, X_label, Y_label)
logistic_regression_time = time.time()
print("--- naive bayes -> logistic regression :%s seconds ---" % (logistic_regression_time - naive_bayes_time))


print ("-------------------------------------------")
print ("\n\n Linear SVM result on doc2vec_embedding: ")
y_pred_svm = linear_svm(X_train, Y_train, X_label, Y_label)
linear_svm_time = time.time()
print("--- logistic regression -> linear svm :%s seconds ---" % (linear_svm_time - logistic_regression_time))
print("--- TOTAL TIME: %s seconds ---" % (time.time() - start_time))

--- start -> preprocessing:18.0743200779 seconds ---


 Naive bayes result on doc2vec_embedding: 
Naive Bayes Model score: 0.484
Naive Bayes Model confusion matrix:
[[213  37]
 [221  29]]
0.696761963957
--- preprocessing -> naive bayes :0.0218510627747 seconds ---
-------------------------------------------


 Logistic Regression result on doc2vec_embedding: 


/Volumes/Barly/Anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Logistic Regression score: 0.472
Logistic Regression confusion matrix:
[[197  53]
 [211  39]]
0.695261293329
--- naive bayes -> logistic regression :16.1554670334 seconds ---
-------------------------------------------


 Linear SVM result on doc2vec_embedding: 
SVC score: 0.474
SVC confusionmatrix:
[[211  39]
 [224  26]]
--- logistic regression -> linear svm :189.580327988 seconds ---
--- TOTAL TIME: 223.832290173 seconds ---


In [6]:
y_pred_nb = y_pred_nb.tolist()
y_pred_lr = y_pred_lr.tolist()
y_pred_svm = y_pred_svm.tolist()

In [7]:
df_test['nb_duplicate'] = y_pred_nb
df_test['lr_duplicate'] = y_pred_lr
df_test['svm_duplicate'] = y_pred_svm

In [8]:
df_test.to_csv("/Volumes/Barly/NLP/Project/5k_testing/Type3/Results/test_doc2vec_classifier_pretrained.csv", index = False)

In [ ]:
# Comparing the cosine similarity between vectors on different thresholds
df_new = pd.read_csv("/Volumes/Barly/NLP/Project/5k_testing/Type3/data/doc2Vec/test_doc2vec_classifier_pretrained.csv")

def conv(elem):
    a = re.compile(r"[-+]?\d*\.\d+|\d+")
    return re.findall(a, elem)[0]

threshold_list = [0.2, 0.29, 0.3,0.35, 0.37, 0.39,0.4,0.5,0.6, 0.7]
df_cos_test = df_new
df_cos_test['cos'] = map(conv, df_cos_test['cos'])
df_cos_test['cos'] = df_cos_test['cos'].apply(pd.to_numeric)

target_names = [0,1]
for threshold in threshold_list:
    
    print "Threshold: ", threshold
    
    df_cos_test['thre_'+str(threshold)] = np.where(df_cos_test['cos']>=threshold, 1, 0)

    print confusion_matrix(df_cos_test['is_duplicate'], df_cos_test['thre_'+str(threshold)]\
                                      , labels=target_names).ravel()
    
    print classification_report(df_cos_test['is_duplicate'], df_cos_test['thre_'+str(threshold)])

    print "Accuracy: " ,accuracy_score(df_cos_test['is_duplicate'], df_cos_test['thre_'+str(threshold)])
    print "\n*******************"